### Implementação Básica de Filtragem Colaborativa baseada em Itens

Filtragem colaborativa é um processo oriundo de sistemas de recomendação, onde o objetivo é filtrar uma lista de itens para sugerir a um usuário o item mais "relevante" dessa lista.

Itens são avaliados como "relevante" a partir de uma análise que busca por similaridades em um histórico de avaliações.

#### 1. Importação de Bibliotecas

In [1]:
import numpy as np

#### 2. Histórico de Avaliações

Usaremos dados de um histórico de avaliações para os processos subsequentes.

Para cada usuário, um array contém as avaliações daquele usuário para cada item de um catálogo:

`usuario = [item0, item1, item2, item3, ...]`

Caso um usuário não tenha avaliado um item, deixamos a avaliação como `0`.

In [2]:
NUM_USUARIOS = 4
NUM_ITENS = 5

usuario0 = [5, 1, 0, 2, 2]
usuario1 = [1, 5, 2, 5, 5]
usuario2 = [2, 0, 3, 5, 4]
usuario3 = [4, 3, 5, 3, 0]


Vamos transformar esses dados em uma matrix (um array 2D).

In [3]:
avaliacoes = [usuario0, usuario1, usuario2, usuario3]

matrix_avaliacoes = np.array(avaliacoes)

In [4]:
print(matrix_avaliacoes)

[[5 1 0 2 2]
 [1 5 2 5 5]
 [2 0 3 5 4]
 [4 3 5 3 0]]


Iremos transpor nossa matrix de avaliações, de modo que linhas serão itens, e colunas serão usuários.

In [5]:
matrix_itens = np.transpose(matrix_avaliacoes)

In [6]:
print(matrix_itens)

[[5 1 2 4]
 [1 5 0 3]
 [0 2 3 5]
 [2 5 5 3]
 [2 5 4 0]]


Isso é necessário. Agora, cada linha contém um vetor de todas avaliações dadas a um item.

Podemos comparar esse vetores para ver o quão similar cada item é.

#### 3. Calculando a similaridade entre dois itens.

Vamos definir uma função que pega dois itens e retorna a similaridade entre eles.

In [7]:
def calcular_similaridade(vetor1: np.ndarray, vetor2: np.ndarray) -> float:
    """
    Calcula a similaridade entre dois vetores a partir do coseno do ângulo entre
    eles.

    Args:
        item1: numpy.ndarray do primeiro vetor
        item2: numpy.ndarray do segundo vetor

    Returns:
        similaridade: float correspondendo a similaridade entre os dois vetores
    """

    dot = np.dot(vetor1, vetor2)
    
    magnitude1 = np.linalg.norm(vetor1)
    magnitude2 = np.linalg.norm(vetor2)

    similaridade = dot / (magnitude1 * magnitude2)

    return similaridade

Com esses materiais, já conseguimos calcular a similaridade entre itens.

In [8]:
for i, item in enumerate(matrix_itens):
    for j, outro_item in enumerate(matrix_itens):
        sim = calcular_similaridade(item, outro_item)
        print(f"Similaridade entre Itens {i} e {j}: {sim:.3g}")

Similaridade entre Itens 0 e 0: 1
Similaridade entre Itens 0 e 1: 0.548
Similaridade entre Itens 0 e 2: 0.67
Similaridade entre Itens 0 e 3: 0.687
Similaridade entre Itens 0 e 4: 0.506
Similaridade entre Itens 1 e 0: 0.548
Similaridade entre Itens 1 e 1: 1
Similaridade entre Itens 1 e 2: 0.686
Similaridade entre Itens 1 e 3: 0.767
Similaridade entre Itens 1 e 4: 0.68
Similaridade entre Itens 2 e 0: 0.67
Similaridade entre Itens 2 e 1: 0.686
Similaridade entre Itens 2 e 2: 1
Similaridade entre Itens 2 e 3: 0.818
Similaridade entre Itens 2 e 4: 0.532
Similaridade entre Itens 3 e 0: 0.687
Similaridade entre Itens 3 e 1: 0.767
Similaridade entre Itens 3 e 2: 0.818
Similaridade entre Itens 3 e 3: 1
Similaridade entre Itens 3 e 4: 0.92
Similaridade entre Itens 4 e 0: 0.506
Similaridade entre Itens 4 e 1: 0.68
Similaridade entre Itens 4 e 2: 0.532
Similaridade entre Itens 4 e 3: 0.92
Similaridade entre Itens 4 e 4: 1


#### 4. Matrix de Similaridades

Podemos imaginar uma outra matrix Itens X Itens, denominada **"Matrix de Similaridades"**, para guardar esses dados de similaridades.

In [9]:
matrix_similaridades = np.empty(NUM_ITENS, dtype=np.ndarray)

for i in range(NUM_ITENS):
    similar = []

    for j in range(NUM_ITENS):
        similar.append(calcular_similaridade(matrix_itens[i], matrix_itens[j]))

    matrix_similaridades[i] = np.array(similar)

matrix_similaridades = np.stack(matrix_similaridades)

In [10]:
print(matrix_similaridades)

[[1.         0.54828926 0.66971082 0.6873098  0.50552503]
 [0.54828926 1.         0.68551062 0.76665188 0.68033605]
 [0.66971082 0.68551062 1.         0.81751912 0.53201592]
 [0.6873098  0.76665188 0.81751912 1.         0.92027908]
 [0.50552503 0.68033605 0.53201592 0.92027908 1.        ]]


Outra maneira mais rápida de calcular essa matrix é a partir dessa linha de código:

In [11]:
matrix_similaridades = np.dot(matrix_itens, matrix_itens.T) / (np.linalg.norm(matrix_itens, axis=1)[:, None] * np.linalg.norm(matrix_itens.T, axis=0))

In [12]:
print(matrix_similaridades)

[[1.         0.54828926 0.66971082 0.6873098  0.50552503]
 [0.54828926 1.         0.68551062 0.76665188 0.68033605]
 [0.66971082 0.68551062 1.         0.81751912 0.53201592]
 [0.6873098  0.76665188 0.81751912 1.         0.92027908]
 [0.50552503 0.68033605 0.53201592 0.92027908 1.        ]]


Agora podemos utilizar essa matrix de similaridades para avaliar a utilidade de um item para um usuário.

#### 5. Avaliação de Utilidade

Vamos definir uma função denominada "Avaliar" que pega um usuário e um item, e retorna a Utilidade daquele item para aquele usuário.

In [13]:
def avaliar(usuario: int, item: int) -> float:
    """"
    Dado um usuário e um item, retorna a utilidade daquele item para o usuário.

    Args:
        usuario: int do id do usuário
        item: int do id do item
        k: int do número de usuarios similares usados na comparação

    Returns:
        utilidade: float correspondente a previsão da utilidade/avaliação do 
                   item para o usuário
    """

    # Número de itens similares utilizados no cálculo da utilidade.
    k = 2

    # Primeiro, pegamos o grau de similaridade de cada item com o item 
    # definido no argumento.
    similaridades = matrix_similaridades[item]

    # Agora precisamos obter o id dos k itens mais similares ao item 
    # definido no argumento que foram avaliados pelo usuário definido 
    # no argumento.

    # Rankear todos itens base no grau de similaridade, e obter o id dos 
    # itens rankeados.
    itens_rankeados = list(np.argsort(similaridades)[::-1][1:])

    # Para cada item, se o usuário definido no argumento não o avaliou,
    # remova-o da lista de itens rankeados.
    for index, item_rankeado in enumerate(itens_rankeados):
        if matrix_avaliacoes[usuario, item_rankeado] == 0:
            itens_rankeados.pop(index)

    # Obter os k itens mais similares que foram avaliados pelo usuário definido 
    # no argumento.
    itens_mais_similares = itens_rankeados[:k]

    # Obter as avaliações desses itens do usuário e o grau de similaridade 
    # de cada um.
    avaliacoes_similares_do_usuario = matrix_avaliacoes[usuario, itens_mais_similares]
    grau_similaridade = matrix_similaridades[item, itens_mais_similares]

    # Fazer uma média ponderada das avaliações, onde os pesos são os graus 
    # de similaridade.
    soma_ponderada = 0
    for avaliacao, grau in zip(avaliacoes_similares_do_usuario, grau_similaridade):
        soma_ponderada += grau * avaliacao 
    soma_pesos = sum(grau_similaridade)
    media_ponderada = soma_ponderada / soma_pesos

    # Definimos a utilidade como a média ponderada.
    utilidade = media_ponderada

    return utilidade

Agora podemos avaliar a utilidade de cada item para cada usuário:

In [14]:
for u in range(NUM_USUARIOS):
    for i in range(NUM_ITENS):
        print()
        print(f"Avaliando item {i} para usuário {u}: {avaliar(u, i):.3g}", end="")


Avaliando item 0 para usuário 0: 1.56
Avaliando item 1 para usuário 0: 2
Avaliando item 2 para usuário 0: 1.54
Avaliando item 3 para usuário 0: 1.55
Avaliando item 4 para usuário 0: 1.57
Avaliando item 0 para usuário 1: 3.52
Avaliando item 1 para usuário 1: 3.58
Avaliando item 2 para usuário 1: 5
Avaliando item 3 para usuário 1: 3.59
Avaliando item 4 para usuário 1: 5
Avaliando item 0 para usuário 2: 4.01
Avaliando item 1 para usuário 2: 4.06
Avaliando item 2 para usuário 2: 3.65
Avaliando item 3 para usuário 2: 3.53
Avaliando item 4 para usuário 2: 4.27
Avaliando item 0 para usuário 3: 3.99
Avaliando item 1 para usuário 3: 3.94
Avaliando item 2 para usuário 3: 3
Avaliando item 3 para usuário 3: 4.03
Avaliando item 4 para usuário 3: 3

Vamos criar um ranqueamento dessas avaliações, e recomendar os itens mais bem ranqueados e desconhecidos pelos usuários. Um item desconhecido é um item que o usuário ainda não avaliou.

Em um cenário com maior disponibilidade de dados, estariamos apenas avaliando os itens que os usuários desconhecem, e criando um ranqueamento a partir disso. Aqui, estou simplificando o processo pois a disponibilidade de dados é menor.

In [20]:
recomendacoes = []

for u in range(NUM_USUARIOS):
    avaliacoes = []
    for i in range(NUM_ITENS):
        avaliacao = avaliar(u, i)
        if matrix_avaliacoes[u, i] != 0:
            continue
        else:
            avaliacoes.append((i, avaliacao))
    
    avaliacoes_ranqueadas = sorted(avaliacoes, key=(lambda x: x[1]))

    recomendacoes.append((u, avaliacoes_ranqueadas))

print("Recomendações\n")
for recomendacao in recomendacoes:
    print(f"Usuário {recomendacao[0]}: ", end='')

    if recomendacao[1]:
        for i in recomendacao[1]:
            print(f"Item {i[0]}", end='')
    
    print("")

Recomendações

Usuário 0: Item 2
Usuário 1: 
Usuário 2: Item 1
Usuário 3: Item 4
